<a href="https://colab.research.google.com/github/AlugubellySaisri/diabetes/blob/main/week%209%20LSA%20tutorial%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.linalg import Vectors
from pyspark.ml.decomposition import SVD
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

ModuleNotFoundError: No module named 'pyspark.ml.decomposition'

In [2]:
!pip install matplotlib-venn

In [3]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [4]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 1.1 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=a3a79f865020d1c988704daf785bfad0353364488d4c638b5f6da1cf98d2a6f7
  Stored in directory: /root/.cache/pip/wheels/32/98/bd/4893d6923dd027f455b250367d402bfd69a6f4416581df46db
Successfully built libarchive


In [5]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [6]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 53.6 MB/s eta 0:00:00


In [14]:
# Initialize Spark Session
spark = SparkSession.builder.master("local[*]").appName("LSA").getOrCreate()

In [8]:
# Sample data (a list of documents)
data = [
    (0, "Spark is a unified analytics engine for big data processing"),
    (1, "Machine learning is a method of data analysis that automates analytical model building"),
    (2, "Deep learning models are built using artificial neural networks"),
    (3, "Data science is an inter-disciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge from data")
]

In [9]:
# Create DataFrame
df = spark.createDataFrame(data, ["id", "text"])

In [10]:
# Tokenization
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered")

In [11]:
# TF-IDF Pipeline
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [18]:
# Define the pipeline
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

pipeline = Pipeline(stages=[tokenizer, stopwords_remover, hashingTF, idf])
model = pipeline.fit(df)
transformed_df = model.transform(df)

In [21]:
# Perform Singular Value Decomposition (SVD)

from pyspark.ml.feature import PCA

# Reduce feature dimension from 'features' to 2 principal components
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(transformed_df)
pca_result = pca_model.transform(transformed_df)

In [25]:
# Show the result
pca_result.select("id", "text", "pca_features").show(truncate=False)


+---+-------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------+
|id |text                                                                                                                                       |pca_features                             |
+---+-------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------+
|0  |Spark is a unified analytics engine for big data processing                                                                                |[-0.15815516172594524,0.6392472064131579]|
|1  |Machine learning is a method of data analysis that automates analytical model building                                                     |[-0.43702113735219617,1.3598575158163462]|
|2  |Deep learning models are built using artificial neural 